# Solutions to Problem Set 1 — Coding Part

**Course:** Data Compression With And Without Deep probabilistic models (Prof. Bamler at University of Tuebingen)

- This notebook shows suggested solutions to the coding part of Problem Set 1, published on 22 April 2022 and discussed on 29 April 2022.
- Download the original problem set from the [course website](https://robamler.github.io/teaching/compress22/).

## Problem 1.2: Binary Heap

This exercise is a preparation for Problem 2.3 below, where we will implement the Huffman coding algorithm.
Our implementation will use a *binary heap*.

### Problem 1.2 (a)

Binary heaps are provided in Python by the `heapq` module.
Read the following demo code, verify that it runs, and make sure you understand what it does.

In [1]:
import heapq

In [2]:
# Heaps in Python are represented as lists
test_heap = []

# To interpret `test_heap` as a binary heap, we mustn't use normal list 
# operations like `test_heap.append(...)`. Instead, to insert an item, we
# have to use the function `heapq.heappush'.
heapq.heappush(test_heap, 5)
heapq.heappush(test_heap, 9)
heapq.heappush(test_heap, 2)
heapq.heappush(test_heap, 8)
print(f"test_heap after five insertions: {test_heap}")

# To extract the smallest item of the heap, we have to call `heapq.heappop`:
smallest = heapq.heappop(test_heap)
print(f"smallest item: {smallest}")
print(f"test_heap after extracting one item: {test_heap}")

# Notice that extracting the smallest item also rearranged the remaining
# items in the list. This is so that when we call `heapq.heappop` again we'll
# again get the (now) smallest item
smallest = heapq.heappop(test_heap)
print(f"smallest item: {smallest}")
print(f"test_heap after extracting two items: {test_heap}")

test_heap after five insertions: [2, 8, 5, 9]
smallest item: 2
test_heap after extracting one item: [5, 8, 9]
smallest item: 5
test_heap after extracting two items: [8, 9]


### Problem 1.2 (b)

Implement the body of the following function `heapsort`.
The function takes an unsorted list of items and it should return a list of the same items in sorted order (from smallest to largest).
Don't use any built-in sorting algorithm.
Instead, call `heapq.heappush` repeatedly to push all items on a heap and then extract them into a sorted list, which you return.

Verify your implementation by running the unit tests below.

In [3]:
def heapsort(unsorted_list):
    # TODO: fill in the function body
    #
    # SUGGESTED SOLUTION:
    # (note: this naive solution takes O(k * log(k)) time to construct the heap,
    # where k = len(unsorted_list). There's also a function `heapq.heapify`,
    # which constructs a heap from a list in only O(k) time.)
    heap = []
    for item in unsorted_list:
        heapq.heappush(heap, item)
    sorted_list = []
    while len(heap) != 0:
        sorted_list.append(heapq.heappop(heap))
    heapq.heapify
    return sorted_list

#### Unit test:

In [4]:
test_data1 = [
    2.73, 8.57, -0.29, -0.50, -0.18, 1.18, 2.71, -0.41, 7.71, 9.52, 
    3.48, 0.74, 7.71, 5.24, 3.85, 2.16, 5.71, 9.91, 0.39, 6.26]
test_data2 = [] # Test the corner case of an empty list.
test_data3 = [3.2] # Test the corner case of a list with only one entry.
test_data4 = [2, 5, 2] # Test the corner case of a list with equal entries.

assert heapsort(test_data1) == sorted(test_data1)
assert heapsort(test_data2) == sorted(test_data2)
assert heapsort(test_data3) == sorted(test_data3)
assert heapsort(test_data4) == sorted(test_data4)

# Problem 1.3: Huffman Coding II: Implementation

In this exercise, we'll implement the Huffman Coding algorithm (Algorithm 2 on the problem set).

We represent bit strings (code words) as lists of boolean values, where `True` represents a "one"-bit and `False` represents a "zero" bit.
Please be aware that this would be an extremely inefficient representation for a real application, but the simplicity of this representation will make it easier to understand the algorithm.

## Problem 1.3 (b): Encoding

We start with a class `HuffmanEncoder` that constructs a Huffman tree which is best suited for *encoding*.
The following is a proposal for how you could represent the Huffman tree.
Feel free to use your own representation if you have a better idea.
This proposal is optimized for *encoding* because it stores only pointers from nodes to their parents.
If you want to use the Huffman tree for efficient *decoding* then you should instead (or in addition) store the inverse pointers from nodes to their children (see part (c) of this problem below).

A Huffman tree represents a code book as a binary tree with the symbols $x \in \mathfrak X = \{0,1,\ldots, |\mathfrak X|-1\}$ at its leaves.
Any binary tree with $|\mathfrak X|$ leaves has a total of $(2|\mathfrak X| - 1)$ nodes (including the leaves), regardless of the specific shape of the tree.
We represent the tree as a list (named `parent_nodes`) of length $(2|\mathfrak X| - 1)$ with the following properties:
- the first $|\mathfrak X|$ entries of the list `parent_nodes` correspond to the leaf nodes, i.e., the symbols $x\in\{0,1,\ldots,|\mathfrak X|-1\}$;
- the remaining $(|\mathfrak X|-1)$ entries of the list `parent_nodes` correspond to non-leaf nodes, with the root node at the very end;
- each entry of the list `parent_nodes` is a tuple `(index, label)` where `index` is an index into `parent_nodes` pointing at the parent of the current node, and `label` is the label of the edge between the node and its parent (which is either `True` or `False`);
- since the root node has no parent, `parent_nodes[len(parent_nodes) - 1]` will always be `None`.

### Your Tasks

- Read and understand the code in the method `__getitem__` below.
  Then complete the constructor (`__init__`) so that it creates the tree structure that `__getitem__` expects (as described above).
- Run the minimal provided unit test to make sure you didn't do any obvious mistake.
- Implement and run additional unit tests. In particular, test edge case like ties or a degenerate probability distribution (that puts all mass on a single symbol.

In [5]:
import heapq # See Problem 2.2.

In [6]:
class HuffmanEncoder:
    """A code book for Huffman coding, optimized for *encoding*."""
    
    def __init__(self, probabilities):
        """Create an optimal prefix code using the Huffman algorithm.
        
        The alphabet is assumed to be of the form
        {0, 1, 2, ..., probabilities.len() - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each symbol
                in the alphabet. The first entry is the probability of symbol `0`,
                the second entry is the probability of symbol `1`, and so on.
                Must be nonnegative and sum to 1 (up to rounding errors) for the
                Huffman algorithm to be correct.
        """
        
        parent_nodes = [None] * len(probabilities)
        
        frontier = []
        # TODO: turn `frontier` into a binary heap of pairs `(probability, symbol)`,
        # representing the set `F` from Algorithm 2 on the problem set.
        # (You may want to refer back to Problem 2.2 for this.)
        #
        # SUGGESTED SOLUTION: (again, using `heapq.heapify` might be more efficient)
        for symbol, probability in enumerate(probabilities):
            heapq.heappush(frontier, (probability, symbol)) # (sorts lexicographically)
        
        while len(frontier) > 1:
            # TODO: implement the rest of the algorithm, as described on the problem set:
            # - update the `frontier` heap by popping off two items and inserting one new item;
            # - add a new node and two edges to the graph (you may find it easiest to
            #   initially append `None` elements to `parent_nodes` and to mutate them later);
            # - feel free to use a different approach if you find it easier to do so.
            #
            # SUGGESTED SOLUTION:
            weight1, symbol1 = heapq.heappop(frontier)
            weight2, symbol2 = heapq.heappop(frontier)
            heapq.heappush(frontier, (weight1 + weight2, len(parent_nodes)))
            parent_nodes[symbol1] = (len(parent_nodes), False)
            parent_nodes[symbol2] = (len(parent_nodes), True)
            parent_nodes.append(None)

        self.parent_nodes = parent_nodes
        
    def __getitem__(self, symbol):
        """Return the code word for a given symbol.

        Args:
            symbol (int): A symbol from the alphabet {0, 1, ..., len-1}.
                
        Returns:
            The code word for `symbol`.
        """
        assert 0 <= symbol and symbol < len(self)
        
        codeword_reverse = []
        index = symbol
        while True:
            parent = self.parent_nodes[index]
            if parent is None:
                # Found root node.
                break
            else:
                index, bit = parent
                codeword_reverse.append(bit)
        
        return list(reversed(codeword_reverse))
    
    def __len__(self):
        """Returns the size of the alphabet."""
        # A binary tree with `N` leaves has `M = 2*N - 1` nodes, regardless of
        # its shape. Therefore, `N = (M + 1) / 2`.
        return (len(self.parent_nodes) + 1) // 2 # ("//" denotes integer division)

#### Unit rests:

In [7]:
def minimal_encoder_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    huffman_encoder = HuffmanEncoder(probabilities)
    for symbol, expected_codeword in expected_codebook.items():
        assert huffman_encoder[symbol] == expected_codeword

minimal_encoder_test()

In [8]:
# TODO: implement more tests, in particular for edge cases

## Problem 1.3 (c): Decoding

The above class `HuffmanEncoder` is a bit awkward to use if we want to *decode* data:
we would always have to construct the entire codebook and then search for the one code word that matches the beginning of our compressed bit string.
There's a smarter way to use a Huffman tree for decoding:
we can simply read the compressed bit string one bit at a time, and then literally follow the path from the root of the Huffman tree to a leaf node that is defined by the beginning of the compressed bit string.

For example, assume our Huffman tree looks as follows:

```text
symbols (leaves): 0   1   2   3
                   \   \ /   /
                    \   *   /
                     \ /   /
                      *   /
                       \ /
                       root
```

and assume that all "`\`" edges have label `False` and all "`/`" edges have label `True`.
Then, if we want to decode the compressed bit string `[False, True, False, False, ...]`,
we would start at the root node, turn left, then turn right, and then turn left to arrive at the leave node for symbol `1`.
Then we return to the root node, turn left, and so on.
This way, we never have to construct the code words for symbols that don't appear in the message.

In order to follow the above procedure, however, we have to store the Huffman tree in a different way than we did in part (b) above because we need to be able to follow edges *from parents to children* rather than the other way around.
The following is a proposal for how you could represent a Huffman tree that is optimized for *decoding* (again, feel free to use your own representation if you have a better idea).
This time, we only store pointers from parents to children.
We thus don't have to store anything for the $|\mathfrak X|$ leaf nodes (which don't have any children).
There are $|\mathfrak X|-1$ non-leaf nodes (including the root node).
We store them in a list with name `child_nodes` of length $|\mathfrak X|-1$, which has the root node at the end.
Each list item is a tuple `(left_child, right_child)` where:
- if the left child is a leaf node $x$ (thus, $0 \leq x < |\mathfrak X|$), then we set $\texttt{left\_child} = x$;
- otherwise, we set $\texttt{left\_child} = |\mathfrak X| + i$ where $i$ is the index into `child_nodes` that addresses the left child of the current node.
- We use the same scheme for `right_child`.

### Your Tasks

- Read and understand the code in the method `decode` below.
  Then complete the constructor (`__init__`) so that it creates the tree structure that `decode` expects (as described above).
- Run the minimal provided unit test to make sure you didn't do any obvious mistake.
- Implement and run additional unit tests. In particular, test edge case like ties or a degenerate probability distribution (that puts all mass on a single symbol.
- Implement a more extensive round-trip test where you encode a long random message using your `HuffmanEncoder` and then decode it with your `HuffmanDecoder`.

In [9]:
class HuffmanDecoder:
    """A code book for Huffman coding, optimized for *decoding*."""
    
    def __init__(self, probabilities):
        """Create an optimal prefix code using the Huffman algorithm.
        
        The alphabet is assumed to be of the form
        {0, 1, 2, ..., probabilities.len() - 1}.

        Args:
            probabilities (list or numpy array): The probabilities of each symbol
                in the alphabet. The first entry is the probability of symbol `0`,
                the second entry is the probability of symbol `1`, and so on.
                Must be nonnegative and sum to 1 (up to rounding errors) for the
                Huffman algorithm to be correct.
        """
        
        child_nodes = []
        
        frontier = []
        # TODO: turn `frontier` into a binary heap of pairs `(probability, symbol)`,
        # as you did in part (b) of this problem.
        #
        # SUGGESTED SOLUTION: (again, using `heapq.heapify` might be more efficient)
        for symbol, probability in enumerate(probabilities):
            heapq.heappush(frontier, (probability, symbol)) # (sorts lexicographically)

        while len(frontier) > 1:
            # TODO: implement the rest of the algorithm, as described on the problem set:
            # - update the `frontier` heap by popping off two items and inserting one new
            #   item, as in part (b) of this problem;
            # - append a tuple to `child_nodes` that points to its two children.
            # - feel free to use a different approach if you find it easier to do so.
            #
            # SUGGESTED SOLUTION:
            weight1, symbol1 = heapq.heappop(frontier)
            weight2, symbol2 = heapq.heappop(frontier)
            new_index = len(probabilities) + len(child_nodes)
            heapq.heappush(frontier, (weight1 + weight2, new_index))
            child_nodes.append((symbol1, symbol2))

        self.child_nodes = child_nodes
        
    def decode(self, encoded):
        """Decode a sequence of symbols.
        
        The method iterates owill consume bits from argument `bit_iterator` until it has
        found a complete code word. Note that this mutates `bit_iterator`
        (because it advances the iterator). Thus, to decode a sequence of
        symbols, simply create an iterator over the compressed bit string
        (e.g., with `bit_iterator = iter(bit_string)` if `bit_string` is a list
        or numpy array of booleans) and then call `decode(bit_iterator)`
        several times in a row.

        Args:
            encoded: A compressed representation of a message, represented as a list of
                boolean values
            
        Yields:
            A sequence of decoded symbols. Thus, you can iterate over the resulting
            symbols as follows

            ```
            huffman_decoder = HuffmanDecoder(...)
            for symbol in huffman_decoder.decode([True, False, False, True, ...]):
                # ... do something with `symbol` ...
            ```
        """

        node = self.child_nodes[-1] # root node is at the end
        alphabet_size = len(self)
        for bit in encoded:
            child_index = node[1 if bit else 0]
            if child_index < alphabet_size:
                # Reached a leaf node, `child_index` is the decoded symbol.
                yield child_index
                node = self.child_nodes[-1] # Return to root node.
            else:
                # We have to shift `child_index` by `alphabet_size` because
                # we don't explicitly store the leaf nodes.
                node = self.child_nodes[child_index - alphabet_size]

    def __len__(self):
        """Returns the size of the alphabet."""
        # A binary tree with `N` leaves has `M = N - 1` non-leaf nodes,
        # regardless of its shape. Therefore, `M = N + 1`.
        return len(self.child_nodes) + 1

#### Unit rests:

In [10]:
def minimal_decoder_test():
    probabilities = [.3, .28, .12, .1, .2]
    expected_codebook = {
        0: [True, True],
        1: [True, False],
        2: [False, True, True],
        3: [False, True, False],
        4: [False, False],
    }
    encoded = [True, False, False, True, True, False, False, True, True]
    huffman_decoder = HuffmanDecoder(probabilities)
    decoded = list(huffman_decoder.decode(encoded))
    assert decoded == [1, 2, 4, 0]

minimal_decoder_test()

In [11]:
# TODO: implement a round-trip test.
#
# PROPOSED SOLUTION:

import numpy as np

rng = np.random.RandomState(seed=123) # always set a random seed so that tests are reproducible

probabilities = [.3, .28, .12, .1, .2]
expected_codebook = {
    0: [True, True],
    1: [True, False],
    2: [False, True, True],
    3: [False, True, False],
    4: [False, False],
}

#### Proposed Round-Trip Tests

We generate $19$ different random probability distributions $p_j$, each operating on a different alphabet $\mathfrak X_j$ with a different size $|\mathfrak X_j| \in \{2, 3, \ldots, 20\}$.
Then, we generate $19$ different messages by drawing $1000$ symbols from each distribution.
We encode each message with its corresponding code book and then decode the resulting bit string and verify that we get back the original message.

In addition, we calculate the entropy of each distribution and the expected code word length of each Huffman code.
We compare these values to the empirical bit rate per symbol that we get in each message.

In [12]:
import numpy as np

In [13]:
rng = np.random.RandomState(seed=123) # always set a random seed, so that you can reproduce and debug any failing tests
distributions = [
    rng.dirichlet(alpha=[1.0] * size) # (a Dirichlet distribution is a distribution over (categorical) distributions)
    for size in range(2, 21)
]
messages = [
    rng.choice(len(distribution), size=1000, p=distribution)
    for distribution in distributions
]
messages[0], messages[1], messages[18] # Look at the messages for debugging purpose.

(array([0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1,
        1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0,
        1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 

In [14]:
def encode(message, distribution):
    encoded = []
    encoder_codebook = HuffmanEncoder(distribution)
    for symbol in message:
        encoded += encoder_codebook[symbol] # append the code word to the encoded bit string
    return encoded

def decode(bitstring, distribution):
    decoder_codebook = HuffmanDecoder(distribution)
    return list(decoder_codebook.decode(bitstring))

for message, distribution in zip(messages, distributions):
    # Encode the message:
    encoder_codebook = HuffmanEncoder(distribution)
    bitstring = []
    for symbol in message:
        bitstring += encoder_codebook[symbol] # append the code word to the encoded bit string

    # Encode the compressed bit string:
    decoder_codebook = HuffmanDecoder(distribution)
    reconstruction = list(decoder_codebook.decode(bitstring))

    # Verify correctness:
    assert np.all(reconstruction == message)

    # Print some statistics:
    entropy = -distribution.dot(np.log2(distribution))
    codewords_lengths = [len(encoder_codebook[symbol]) for symbol in range(len(distribution))]
    expected_codeword_length = distribution.dot(codewords_lengths)
    bits_per_symbol = len(bitstring) / len(message)
    print(
        f"alphabet size{len(distribution): 3d}: entropy = {entropy:.2f}; " +
        f"L = {expected_codeword_length:.2f}; " +
        f"empirical bitrate per symbol = {bits_per_symbol:.2f}"
    )

print("Done.")

alphabet size  2: entropy = 0.76; L = 1.00; empirical bitrate per symbol = 1.00
alphabet size  3: entropy = 1.36; L = 1.45; empirical bitrate per symbol = 1.45
alphabet size  4: entropy = 1.52; L = 1.57; empirical bitrate per symbol = 1.58
alphabet size  5: entropy = 1.95; L = 2.05; empirical bitrate per symbol = 2.04
alphabet size  6: entropy = 2.30; L = 2.34; empirical bitrate per symbol = 2.37
alphabet size  7: entropy = 2.64; L = 2.68; empirical bitrate per symbol = 2.68
alphabet size  8: entropy = 2.80; L = 2.84; empirical bitrate per symbol = 2.81
alphabet size  9: entropy = 2.65; L = 2.70; empirical bitrate per symbol = 2.72
alphabet size 10: entropy = 2.82; L = 2.89; empirical bitrate per symbol = 2.88
alphabet size 11: entropy = 3.28; L = 3.33; empirical bitrate per symbol = 3.29
alphabet size 12: entropy = 3.05; L = 3.08; empirical bitrate per symbol = 3.09
alphabet size 13: entropy = 3.17; L = 3.21; empirical bitrate per symbol = 3.21
alphabet size 14: entropy = 3.03; L = 3.